In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats, sparse, spatial
from qiime2 import Artifact

In [162]:
data = Artifact.load("../Chemobrain/200608_Elsayed_Lowry-1101/tax-table.qza").view(pd.DataFrame)

In [175]:
datamat = np.array(data)
datamat /= datamat.sum(axis=1, keepdims=True)

data = pd.DataFrame(datamat,
                    index=data.index,
                    columns=data.columns)

In [236]:
def get_density(envdata):
    #generate a numpy matrix for use later
    datamat = np.array(envdata)

    #quantify the density
    density = ( np.count_nonzero(datamat) / float(datamat.size) )

    return density


def fit_to_microbiome(envdata):
    #generate a numpy matrix for use later
    datamat = np.array(envdata)

    #quantify the general parameters to start populate with
    #lognormal distributions are frequently used for microbiome data
    s, loc, scale = stats.lognorm.fit(datamat[np.nonzero(datamat)])
    rvs = stats.lognorm(s=s, loc=loc, scale=scale).rvs
    
    return rvs



def populate_initial(n_features=1, n_cells=1, data_rvs=None):

    #create a sparse matrix with a lognormal distribution
    bcells = sparse.random(n_cells, n_features, density=density, 
              random_state=42, data_rvs=rvs).A

    #convert to relative abundances
    bcells /= bcells.sum(axis=1, keepdims=True)

    return bcells







  #  def bcell_index(bcells, data):
  #      bcells
        
    
 #   def __init__(self):
        #iszero = np.random.randint(2, size=n)
        #self.data = np.random.default_rng().negative_binomial(20,0.9,n) * iszero
        #self.data = self.data/sum(self.data)
        
  #      self.data = []
        
        

In [237]:
density = get_density(data)
rvs = fit_to_microbiome(data)

bcells = populate_initial(n_features=data.shape[1], n_cells=100, data_rvs=rvs)

In [238]:
bcelldf = pd.DataFrame(bcells)

In [239]:
pd.DataFrame(spatial.distance.cdist(bcells, data, metric='braycurtis')).min()

0     0.911675
1     0.825714
2     0.903402
3     0.863698
4     0.807302
5     0.923410
6     0.860647
7     0.800553
8     0.932615
9     0.905326
10    0.807760
11    0.907806
12    0.869651
13    0.881497
14    0.902694
15    0.920193
16    0.887446
17    0.926104
18    0.884955
19    0.928544
20    0.942092
21    0.896072
22    0.910446
23    0.877220
24    0.894351
25    0.876074
26    0.837933
27    0.903126
28    0.876488
29    0.858862
30    0.955826
31    0.888825
32    0.921316
dtype: float64

In [301]:
%%time

successful_attempts = []
successful_indexes = []

successful_bcells = np.empty((0,data.shape[1]))
attempt=0

x = immune_system()

density = get_density(data)
rvs = fit_to_microbiome(data)


while len(successful_attempts) < 10:
    attempt +=1
    bcells = populate_initial(n_features=data.shape[1], n_cells=100, data_rvs=rvs)
                                        
                                        #BCELLS MUST COME FIRST HERE - maybe make a function for this?
    matches = sum(spatial.distance.cdist(bcells, data, metric='braycurtis').flatten() < 0.5)
    
    for match in range(matches):
        successful_attempts.append(attempt)
    
    if matches > 0:
        good_ones = np.where( spatial.distance.cdist(bcells, data, metric='braycurtis') < 0.5 )
        
        successful_indexes += list(good_ones)    
        
        successful_bcells = np.vstack((successful_bcells, 
                                       bcells[good_ones[0]]))
                #using [0] in the line above grabs only the b cell's index, not the sample data

CPU times: user 18.5 s, sys: 120 ms, total: 18.6 s
Wall time: 18.9 s


In [297]:
successful_attempts

[7, 7, 32, 165, 224, 348, 348, 348, 517, 570]

In [298]:
successful_indexes

[array([88, 88]),
 array([ 0, 17]),
 array([24]),
 array([28]),
 array([38]),
 array([28]),
 array([62]),
 array([28]),
 array([92, 92, 92]),
 array([20, 30, 32]),
 array([19]),
 array([8]),
 array([29]),
 array([8])]

In [299]:
successful_bcells

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00046026, 0.00257053, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.00176552, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00159522,
        0.00143594]])

In [272]:
np.where(spatial.distance.cdist(bcells, data, metric='braycurtis') < 0.5)[0][0]

23

In [227]:
sum(spatial.distance.cdist(bcell, data, metric='braycurtis').flatten() < 0.5)

0

In [300]:
bcells[np.where( spatial.distance.cdist(bcells, data, metric='braycurtis') < 0.5 )[0]]

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        8.02187400e-04, 0.00000000e+00, 0.00000000e+00, 6.45131886e-04,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 7.48708822e-04, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        4.23488428e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 3.56443599e-04, 1.42719288e-03,
        0.00000000e+00, 5.33290340e-04, 2.99307305e-03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.47289852e-04,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000